In [71]:
import pandas as pd
import numpy as np

In [72]:
# read in datasets 
hrsfgcp = pd.read_stata('/Users/novak/Columbia-IPHD Dropbox/CU_IPHD/HCAP/data/source/hrshcap/interim-HcapHarmoniz-305-v2-20230725-hrshcap.dta')
hc16_r = pd.read_stata('/Users/novak/Columbia-IPHD Dropbox/CU_IPHD/Data/HRS/cognition/2016HCAP/HC16sta/hc16hp_r.dta')
hc16_i = pd.read_stata('/Users/novak/Columbia-IPHD Dropbox/CU_IPHD/Data/HRS/cognition/2016HCAP/HC16sta/hc16hp_i.dta')
trk2020tr_r = pd.read_stata('/Users/novak/Columbia-IPHD Dropbox/CU_IPHD/ImAgingData/data/source/trackerfile/trk2020v1/trk2020tr_r.dta')

In [73]:
# preprocess datasets

## preprocess trk2020tr_r
trk2020tr_r = trk2020tr_r.rename(columns=str.lower)

## preprocess hrsfgcp
hrsfgcp = hrsfgcp[hrsfgcp['study'] == 1]
hrsfgcp['hhidpn'] = hrsfgcp['id_hrs']
print(hrsfgcp[hrsfgcp.duplicated('hhidpn', keep=False)])
hrsfgcp['hhidpn'] = pd.to_numeric(hrsfgcp['hhidpn'], errors='coerce')
hrsfgcp = hrsfgcp[['hhidpn', 'id_hrs', 'fgcp', 'fmem', 'fexf', 'flang', 'forient']]


## merge datasets 
merged_data = pd.merge(hc16_i, hc16_r, on=['hhid', 'pn'], how='inner')
merged_data.columns = merged_data.columns.str.lower()

final_merged_data = pd.merge(merged_data, trk2020tr_r, on=['hhid', 'pn'], how='inner')
final_merged_data.columns = final_merged_data.columns.str.lower()

final_merged_data

Empty DataFrame
Columns: [study, id_hrs, fgcp, fgcp_bayes3, fexf, fexf_bayes3, flang, flang_bayes3, forient, forient_bayes3, fmem, fmem_bayes3, hhidpn]
Index: []


,hhid,pn,inf1iwmonth,inf1iwyear,inf1consent,inf1intro,inf1age,inf1gender,inf1educ,inf1rel,...,rrescode,rsubhh,rsubhhiw,rwebcontrol,rwgthh,rwgtr,rwgtrnh,rwhy0hwt,rwhy0rwt,version
0,010004,040,2.0,2017.0,1.0,1.0,71.0,1.0,12.0,1.0,...,1001.0,1,1.0,5.0,6910.0,6967.0,NaN,0.0,0.0,1.0
1,010038,010,7.0,2016.0,1.0,1.0,73.0,2.0,16.0,1.0,...,1001.0,0,1.0,1.0,7083.0,7475.0,NaN,0.0,0.0,1.0
2,010059,020,1.0,2017.0,1.0,1.0,88.0,1.0,17.0,1.0,...,1001.0,0,1.0,5.0,10995.0,11525.0,NaN,0.0,0.0,1.0
3,010075,020,5.0,2017.0,1.0,1.0,46.0,2.0,17.0,2.0,...,1001.0,0,1.0,5.0,2616.0,3756.0,NaN,0.0,0.0,1.0
4,010106,010,10.0,2016.0,1.0,1.0,88.0,2.0,12.0,97.0,...,1001.0,0,1.0,5.0,7544.0,8281.0,NaN,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3491,920363,010,5.0,2017.0,1.0,1.0,77.0,2.0,7.0,1.0,...,6001.0,0,5.0,5.0,0.0,0.0,NaN,5.0,5.0,1.0
3492,920557,020,10.0,2017.0,1.0,1.0,60.0,2.0,13.0,1.0,...,9081.0,3,5.0,5.0,0.0,0.0,NaN,1.0,1.0,1.0
3493,920697,010,12.0,2016.0,1.0,1.0,63.0,2.0,13.0,6.0,...,1001.0,0,1.0,5.0,961.0,924.0,NaN,0.0,0.0,1.0
3494,920760,020,12.0,2016.0,1.0,1.0,67.0,2.0,12.0,1.0,...,1001.0,2,1.0,5.0,1124.0,1310.0,NaN,0.0,0.0,1.0


In [74]:
# 1. Sample selection;

data = final_merged_data[(final_merged_data['r1recordediw'] == 1) & (final_merged_data['inf1consent'] == 1)]

data['hhidpn'] = data['hhid'].astype(str) + data['pn'].astype(str)
data['hhidpn'] = pd.to_numeric(data['hhidpn'], errors='coerce')


data = pd.merge(data, hrsfgcp, on='hhidpn', how='inner')

data['hrs'] = 1
data['mex'] = 0


data['wgt'] = data['hcap16wgtr'].copy()

data.head(10)

/var/folders/nh/hjw01w5j4k7csj4x6k02jv000000gn/T/ipykernel_27106/921874589.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['hhidpn'] = data['hhid'].astype(str) + data['pn'].astype(str)
/var/folders/nh/hjw01w5j4k7csj4x6k02jv000000gn/T/ipykernel_27106/921874589.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['hhidpn'] = pd.to_numeric(data['hhidpn'], errors='coerce')


,hhid,pn,inf1iwmonth,inf1iwyear,inf1consent,inf1intro,inf1age,inf1gender,inf1educ,inf1rel,...,hhidpn,id_hrs,fgcp,fmem,fexf,flang,forient,hrs,mex,wgt
0,010004,040,2.0,2017.0,1.0,1.0,71.0,1.0,12.0,1.0,...,10004040,10004040,1.290,1.412,1.030,0.268,0.423,1,0,17148
1,010038,010,7.0,2016.0,1.0,1.0,73.0,2.0,16.0,1.0,...,10038010,10038010,0.937,0.112,1.113,1.300,0.423,1,0,16431
2,010059,020,1.0,2017.0,1.0,1.0,88.0,1.0,17.0,1.0,...,10059020,10059020,0.443,0.942,0.029,0.107,-0.332,1,0,18512
3,010075,020,5.0,2017.0,1.0,1.0,46.0,2.0,17.0,2.0,...,10075020,10075020,-1.360,-0.841,-1.599,-0.848,0.400,1,0,9209
4,010106,010,10.0,2016.0,1.0,1.0,88.0,2.0,12.0,97.0,...,10106010,10106010,1.231,1.073,1.089,1.390,0.423,1,0,10982
5,010210,020,4.0,2017.0,1.0,1.0,44.0,2.0,12.0,2.0,...,10210020,10210020,-2.313,-2.013,-2.121,-1.322,-2.379,1,0,6364
6,010325,020,11.0,2016.0,1.0,1.0,55.0,1.0,13.0,2.0,...,10325020,10325020,1.056,0.994,0.813,0.601,0.423,1,0,13232
7,010451,010,7.0,2016.0,1.0,1.0,76.0,1.0,16.0,1.0,...,10451010,10451010,0.634,0.690,0.461,-0.118,0.423,1,0,20299
8,010475,010,5.0,2017.0,1.0,1.0,52.0,2.0,17.0,2.0,...,10475010,10475010,-3.132,-2.376,-3.050,-2.349,-3.085,1,0,16210
9,010477,010,8.0,2016.0,1.0,1.0,80.0,2.0,8.0,1.0,...,10477010,10477010,0.411,-0.456,0.916,0.265,0.423,1,0,17360


In [75]:
#2. Construct variables;

# Step 1: Construct Respondent Characteristics
# Drop the existing 'rage' column if it exists
if 'rage' in data.columns:
    data.drop(columns=['rage'], inplace=True)

# Clone variable 'rage' from 'inf1riwage'
data['rage'] = data['inf1riwage']

# Gender
data['rfemale'] = (data['gender'] == 2).astype(int)
data['rmale'] = (data['gender'] == 1).astype(int)

# Education
data['reduc'] = data['schlyrs'].where(data['schlyrs'] <= 17)
data['reduc'] = data['reduc'].mask((data['schlyrs'] == 99) & (data['degree'] == 4), 16)
data['reduc'] = data['reduc'].mask((data['schlyrs'] == 99) & (data['degree'] == 2), 12)
data['reduc'] = data['reduc'].mask((data['schlyrs'] == 99) & (data['degree'] == 0), 6)

# Missing values for reduc
data['reduc'] = data['reduc'].mask((data['degree'] == 9) & (data['schlyrs'] == 99))

# Display a summary of the 'reduc' variable
print(data['reduc'].describe())

# Step 2: Construct Informant Characteristics
# Age
data['iage'] = data['inf1age']
data['iage'] = data['iage'].mask(data['inf1age'].isin([998, 999]), None)

# Gender
data['ifemale'] = (data['inf1gender'] == 2).astype(int)
data['imale'] = (data['inf1gender'] == 1).astype(int)

# Education
data['ieduc'] = data['inf1educ']
data['ieduc'] = data['ieduc'].mask(data['inf1educ'].isin([998, 999]), None)

# Step 3: Construct R-I Relationships
# Relationship
data['relation'] = None
data['relation'] = data['relation'].mask((data['inf1rel'] == 1) | (data['inf1reloth_a'] == 3), 1)
data['relation'] = data['relation'].mask(data['inf1rel'] == 2, 2)
data['relation'] = data['relation'].mask(data['inf1rel'].isin([3, 4, 5]) | (data['inf1reloth_a'] == 1), 3)
data['relation'] = data['relation'].mask(data['inf1rel'].isin([6, 7, 8]) | (data['inf1reloth_a'] == 2), 4)

# Indicators for relationship types
data['ispouse'] = (data['relation'] == 1).astype(int)
data['ichild'] = (data['relation'] == 2).astype(int)
data['iothfam'] = (data['relation'] == 3).astype(int)
data['inonfam'] = (data['relation'] == 4).astype(int)

# Co-residence
data['coresi'] = ((data['inf1freqsee'] == 1) | ((data['inf1freqsee'] == 2) & (data['ispouse'] == 1))).astype(int)

# Add labels for clarity in visualization or documentation (optional)
# data['ifemale'] = data['ifemale'].astype('category').cat.rename_categories({1: "female informant"})
# data['imale'] = data['imale'].astype('category').cat.rename_categories({1: "male informant"})
# data['coresi'] = data['coresi'].astype('category').cat.rename_categories({1: "co-resident with participant"})


count    2965.000000
mean       12.812479
std         3.165996
min         0.000000
25%        12.000000
50%        12.000000
75%        16.000000
max        17.000000
Name: reduc, dtype: float64


In [76]:
#3. CSID variables;

# ** 2.0 CSI-D
# ** (1)mi_q2_16 = inf1csidmental: general decline in mental functioning 
# ** (2)mi_q3_16 = inf1csidmemory: difficulty remembering things  
# ** (3)mi_q4_16 = inf1csidput: forgets where put things 
# ** (4)mi_q5_16 = inf1csidkept: forgets where things kept
# ** (5)mi_q6_16 = inf1csidfrdname: forgets friends names (1/2)
# ** (6)mi_q7_16 = inf1csidfamname: forgets family member names (1/2)
# ** (7)mi_q8_16 = inf1csidconvers: forgets thoughts in middle of conversation (1/2)
# ** (8)mi_q9_16 = inf1csidwordfind: difficulty finding the right words (1/2)
# ** (9)mi_q10_16 = inf1csidwordwrg:  uses wrong words (1/2)
# ** (10)mi_q11_16 = inf1csidpast: talks about past not present (1/2)
# ** (11)mi_q12_16 = inf1csidlastsee: forgets when last saw informant (1/2)
# ** (12)mi_q13_16 = inf1csidlastday: forgets what happend yesterday 
# ** (13)mi_q14_16 = inf1csidorient: forgets where s/he is 
# ** (14)mi_q15_16 = inf1csidlostout: gets lost in community
# ** (15)mi_q16_16 = inf1csidlostin: gets lost at home
# ** (16)mi_q17_16 = inf1chores: difficulty with household chores
# ** (17)mi_q18_16 = inf1hobby: Loss of skill or hobby/Stopped doing activities/hobbies
# ** (18)mi_q19_16 = inf1money: difficulty handling money
# ** (19)mi_q20_16 = inf1change: difficulty adjusting to routine change 
# ** (20)mi_q22_16 = inf1bl2feed: ability to feed self

csid_vars = [
    "inf1csidmental", "inf1csidmemory", "inf1csidput", "inf1csidkept",
    "inf1csidfrdname", "inf1csidfamname", "inf1csidconvers", "inf1csidwordfind",
    "inf1csidwordwrg", "inf1csidpast", "inf1csidlastsee", "inf1csidlastday",
    "inf1csidorient", "inf1csidlostout", "inf1csidlostin", "inf1chores",
    "inf1hobby", "inf1money", "inf1change"
]

# Rename and recode each CSID variable
for csid in csid_vars:
    new_name = f"b{csid}"
    data[new_name] = data[csid].replace({5: 0, 1: 1, 2: 1}).replace({8: np.nan, 9: np.nan})

# Special case for inf1bl2feed
data['binf1bl2feed'] = data['inf1bl2feed'].replace({1: 0, 2: 1, 4: 1}).replace({8: np.nan, 9: np.nan})

### Generate Missing Indicator
data['mis_csid'] = 0
for csid in [f"b{var}" for var in csid_vars] + ['binf1bl2feed']:
    data.loc[data[csid].isna(), 'mis_csid'] = 1

### Calculate Total CSID Score
# Assuming all binary CSID variables are prefixed with 'b'
csid_columns = [col for col in data if col.startswith('b') and 'csid' in col]
data['bcsid'] = data.loc[data['mis_csid'] == 0, csid_columns].sum(axis=1)

### Drop Rows with Missing CSID
data = data[data['mis_csid'] == 0]

### Frequency Table for Each CSID Item
# Generating frequency tables for each variable and saving them can be done as follows:
# Note: Adjust the file path and format as needed.
for var in csid_columns:
    frequency_table = data[var].value_counts(normalize=True)
    print(f"Frequency table for {var}:\n{frequency_table}\n")

Frequency table for binf1csidmental:
binf1csidmental
0.0    0.692178
1.0    0.307822
Name: proportion, dtype: float64

Frequency table for binf1csidmemory:
binf1csidmemory
0.0    0.853675
1.0    0.146325
Name: proportion, dtype: float64

Frequency table for binf1csidput:
binf1csidput
1.0    0.701955
0.0    0.298045
Name: proportion, dtype: float64

Frequency table for binf1csidkept:
binf1csidkept
0.0    0.682401
1.0    0.317599
Name: proportion, dtype: float64

Frequency table for binf1csidfrdname:
binf1csidfrdname
0.0    0.722185
1.0    0.277815
Name: proportion, dtype: float64

Frequency table for binf1csidfamname:
binf1csidfamname
0.0    0.892785
1.0    0.107215
Name: proportion, dtype: float64

Frequency table for binf1csidconvers:
binf1csidconvers
0.0    0.57822
1.0    0.42178
Name: proportion, dtype: float64

Frequency table for binf1csidwordfind:
binf1csidwordfind
0.0    0.698584
1.0    0.301416
Name: proportion, dtype: float64

Frequency table for binf1csidwordwrg:
binf1csidwor

In [77]:
# Orientation to time
data['r1time'] = data['r1orient_time'].where(data['r1orient_t5'] != 1, data['r1orient_time'] - 1)

# Orientation to place (state)
data['r1state'] = data['riorient_p1'].replace({1: 1, 5: 0, 98: 0, 97: np.nan, 99: np.nan})

# Word recall: immediate and delayed
data['r1imrc3'] = data['r1imrc3'].where(data['r1imrc3'] <= 3, 0)
data['r1dlrc3'] = data['r1dlrc3'].where(data['r1dlrc3'] <= 3, 0)

# Backwards (counting and spelling backwards)
backwards_vars = ['r1backwards1', 'r1backwards2', 'r1backwards3', 'r1backwards4', 'r1backwards5']
for var in backwards_vars:
    data[var] = data[var].replace({1: 1, 5: 0, 98: 0, 97: np.nan, 99: np.nan})
data['r1backwards'] = data[backwards_vars].sum(axis=1, min_count=1)  # sum, skip NaN

# Object names
object_vars = ['r1object1', 'r1object2']
for var in object_vars:
    data[var] = data[var].replace({1: 1, 5: 0, 97: np.nan, 99: np.nan})
data['r1obj'] = data[object_vars].sum(axis=1, min_count=1)

# Repeat a sentence
data['r1rptsen'] = data['r1repeat'].replace({1: 1, 5: 0, 98: 0, 97: np.nan, 99: np.nan})

# Follow written command
data['r1combfol'] = data['r1combfol'].replace({1: 1, 2: 1, 5: 0, 98: 0, 97: np.nan, 99: np.nan})

# Executive function
exec_vars = ['r1execu1', 'r1execu2', 'r1execu3']
for var in exec_vars:
    data[var] = data[var].replace({1: 1, 5: 0, 98: 0, 97: np.nan, 99: np.nan})
data['r1exf'] = data[exec_vars].sum(axis=1, min_count=1)

# Write a sentence
data['r1senten'] = data['r1senten'].replace({1: 1, 2: 1, 5: 0, 98: 0, 97: np.nan, 99: np.nan})

# Draw a picture
data['r1draw'] = data['r1draw'].replace({1: 1, 5: 0, 98: 0, 97: np.nan, 99: np.nan})

# MMSE Total Score
mmse_vars = ['r1time', 'r1state', 'r1imrc3', 'r1dlrc3', 'r1backwards', 'r1obj', 'r1rptsen', 'r1combfol', 'r1exf', 'r1senten', 'r1draw']
data['mmse'] = data[mmse_vars].sum(axis=1, min_count=1)

# Keeping only necessary variables and saving

keep_vars = [
    'hhid', 'pn', 'id_hrs', 'hrs', 'mex', 'wgt',
    'binf1csidmental', 'binf1csidmemory', 'binf1csidput', 'binf1csidkept',
    'binf1csidfrdname', 'binf1csidfamname', 'binf1csidconvers', 'binf1csidwordfind',
    'binf1csidwordwrg', 'binf1csidpast', 'binf1csidlastsee', 'binf1csidlastday',
    'binf1csidorient', 'binf1csidlostout', 'binf1csidlostin', 'binf1chores',
    'binf1hobby', 'binf1money', 'binf1change', 'binf1bl2feed', 'bcsid',
    'fgcp', 'fmem', 'fexf', 'flang', 'forient', 'mmse',
    'rage', 'rfemale', 'reduc', 'iage', 'ifemale', 'ieduc',
    'relation', 'ispouse', 'ichild', 'iothfam', 'inonfam', 'coresi'
]

# Filter the DataFrame to keep only the specified columns
data = data[keep_vars]


# Saving the DataFrame
# Saving the DataFrame as a CSV file
# data.to_csv('/Users/novak/Columbia-IPHD Dropbox/CU_IPHD/HCAP/python file/hrs-hcap.csv', index=False)



In [78]:
# List of variables
variables = ['rage', 'rfemale', 'reduc', 'iage', 'ifemale', 'ieduc', 
             'coresi', 'ispouse', 'ichild', 'iothfam', 'inonfam']

# Create an empty DataFrame to store the summary statistics
summary_table = pd.DataFrame()

# Loop through each variable and calculate the summary statistics
for var in variables:
    summary_stats = data[var].describe()
    summary_table[var] = summary_stats

# Display the summary table
summary_table


,rage,rfemale,reduc,iage,ifemale,ieduc,coresi,ispouse,ichild,iothfam,inonfam
count,2966.000000,2966.000000,2965.000000,2965.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000
mean,76.195887,0.599123,12.812479,64.271501,0.673635,14.755226,0.492245,0.456170,0.262643,0.133513,0.147674
std,7.500089,0.490159,3.165996,14.289663,0.468962,10.564254,0.500024,0.498159,0.440144,0.340186,0.354836
min,65.000000,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,70.000000,0.000000,12.000000,56.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,76.000000,1.000000,12.000000,66.000000,1.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,81.000000,1.000000,16.000000,75.000000,1.000000,16.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,102.000000,1.000000,17.000000,100.000000,1.000000,98.000000,1.000000,1.000000,1.000000,1.000000,1.000000
